In [1]:
import pandas as pd
import copy
from gekko import GEKKO
import numpy as np
import os

In [2]:
m = GEKKO(remote=False)

m.options.IMODE = 3

In [3]:
assets_obs = np.array([
    [0.00907849579534513, -0.00695985093505983, 0.0084299761604594, 0.00460637330420661, -0.0166407210123416, -0.0308227697859888, -0.0159197895240186, -0.0127675605999023, 0.000901306956012888, 0, -0.00904983551991793, 0.00724640852076725, 0.00292891952088526, -0.0119951647586384, -0.00433543177644596, 0.00660971067348635, -0.00958692574332696, 0.0244688238460112, 0.0120134926488238, -0.00576626126438815, -0.00311873721265473, -0.0189194832461658, -0.0285997352356136, -0.108902370498538, -0.00996337555082483, -0.011395379703615, 0.0171757426191143, -0.00472689955760422, 0.0138546197303906, -0.017282441148748, 0.010248414491368, 0.0260626285757702, -0.00280576641150291, 0.00102118977472798, 0, -0.023232846722337, 0.0111674969191731, -0.0214051503199956, 0.0118034157248844, -0.0089052448121078, -0.0151096313917661, -0.00858374369139144, -0.00648650922960677, 0.00890814367013811, -0.00945059011685392, 0.00189727662279466, -0.00325468147354204, -0.0189228188421451, 0.00276472390648269, -0.000828614879616203, 0.0196996794662562, -0.000542005433322949, -0.00325821629540358, -0.0214411732222413, 0.00664638281604644, -0.00859326147298701, 0.00222469502211111, -0.00892863074430132, -0.0059030389262782, -0.00622173952725283, ],
    [0.00303785911242742, 0.00151547582990027, 0.0127974652583047, 0.0190092718209104, 0.0364946864301199, -0.00971090554568595, -0.0136678898182987, -0.00217154254694342, -0.00363651498288239, 0, -0.00291672801750322, 0.00364711784480473, 0.00773265466471527, -0.0290662641433834, 0.00395458691147439, -0.0111698804689666, 0.0513319004579947, 0.0269080669484678, -0.00741373647419926, -0.00140077843916497, 0.00487963349002651, -0.00115827415580574, -0.0159049272751082, 0.00164914589485562, -0.00164914589485567, -0.00733339421955797, -0.00118689660372341, 0.00213440895878598, 0.00756763572455059, 0.00843837240378185, 0.0207943441041438, 0.00956052963076143, 0.00474620545879126, -0.00045239700493063, 0, -0.00792650286859609, -0.0020450391166462, -0.0107647066423052, 0.0119018841383533, 0.011310615413212, -0.00609335250098918, -0.00408137712007475, -0.0128068063784102, 0.00779731980213844, -0.00228927653251101, -0.00344137074074704, -0.00114815814918597, -0.0066893107527231, 0.016761469094511, 0.0227331700639883, 0.00510375629970297, 0.00199266340696577, -0.00553933360593972, 0.00399085554260851, -0.020800099265282, -0.00702904793497261, 0.00363681652749544, -0.00500284917723943, -0.00022864764544903, -0.00939099877556102, ],
    [0.0231043683454177, 0.0417104885813702, -0.0095182158209761, 0.0160098290574834, -0.0172060014489138, -0.00660464724480478, -0.00301659353942568, -0.0133821948406982, 0.0121730048505591, 0, -0.00728600673093332, 0.0211041899519374, 0.0534389621555603, -0.00226500663085212, -0.00113443007061176, -0.00284171830902758, -0.0318037790537778, -0.00885222658735119, 0.00414079266603139, 0.0035356548323078, 0.00235017734495367, 0.00643465221077915, -0.00291971010333475, 0.0127834396256749, -0.0046296378987421, -0.00348634867943777, 0.00406622692125935, -0.00464846856798463, 0, -0.0128958410829249, -0.00236267091597039, 0.0117579249936131, 0.020822036241863, 0.0136443276487771, 0, -0.0125001627642315, -0.00401031760615392, -0.00114876520387337, 0.0102916860365475, 0.00454030291878301, -0.0205956936720807, 0.0114943794257352, -0.0207859677598346, 0.0098694560184841, -0.0174829627803475, 0.00702990592825815, -0.00175284882741436, 0.0110498361865849, -0.00405680069561443, -0.00816331063916098, -0.0052832526765831, 0.0157667499714412, 0.018370206601749, -0.000568990058025087, -0.0247779890293721, -0.0117372239457254, 0.00647251450561752, -0.00176108058429767, -0.00944517057151865, 0.00414324357133459, ],
]).T
n = assets_obs.shape[1]
P = np.asmatrix(np.mean(assets_obs, axis=0))
C = np.cov(assets_obs.T)
m.options.solver = 1

u = m.Array(m.Var, n, lb=0, ub=1, integer=True)
v = m.Array(m.Var, n, lb=0, ub=1, integer=True)
y = m.Array(m.Var, n, lb=0, ub=1, integer=True)

l_plus = m.Array(m.Var, n)
for i in l_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

l_minus = m.Array(m.Var, n)
for i in l_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_plus = m.Array(m.Var, n)
for i in s_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_minus = m.Array(m.Var, n)
for i in s_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1
    
    

In [4]:
for i in range(n):
    m.Maximize(P.item(i) * (l_plus[i] - l_minus[i] - s_plus[i] + s_minus[i]))
    m.Minimize(s_plus[i] - s_minus[i])
    m.Minimize( 0.001*l_plus[i] +0.001*l_minus[i]+ 0.001*s_plus[i]+0.001*s_minus[i])
    m.Equation(m.sum([(l_plus[i] - l_minus[i] +s_plus[i] - s_minus[i]\
                  + 0.001 * l_plus[i] + 0.001 * l_minus[i] \
                  + 0.001 * s_plus[i] + 0.001 * s_minus[i]) \
                 for i in range(n)]) == 1)
m.solve(disp=True,debug=0)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  3
   Constants    :  0
   Variables    :  33
   Intermediates:  0
   Connections  :  12
   Equations    :  21
   Residuals    :  21
 
 Number of state variables:    33
 Number of total equations: -  15
 Number of slack variables: -  0
 ---------------------------------------
 Degrees of freedom       :    18
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    0 Obj: -2.00E+00 Gap:  0.00E+00
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :  0.014899999999999997 sec
 Objective      :  -2.0006

In [5]:
# LP and LM can not be exists at the same time so does SP and SM


print('LP', [x for x in l_plus])
print('LM', [x for x in l_minus])
print('SP', [x for x in s_plus])
print('SM', [x for x in s_minus])

LP [[1.0], [1.0], [1.0]]
LM [[0.0], [0.0], [0.0]]
SP [[0.0], [0.0], [0.0]]
SM [[0.005005005005], [1.0], [1.0]]


In [12]:
'''
教授給的code第一段可行解, w_minus以 (SP - SM)表示, 加入w_plus, w_plus以 (LP - LM)
'''
assets_obs = np.array([
    [0.00907849579534513, -0.00695985093505983, 0.0084299761604594, 0.00460637330420661, -0.0166407210123416, -0.0308227697859888, -0.0159197895240186, -0.0127675605999023, 0.000901306956012888, 0, -0.00904983551991793, 0.00724640852076725, 0.00292891952088526, -0.0119951647586384, -0.00433543177644596, 0.00660971067348635, -0.00958692574332696, 0.0244688238460112, 0.0120134926488238, -0.00576626126438815, -0.00311873721265473, -0.0189194832461658, -0.0285997352356136, -0.108902370498538, -0.00996337555082483, -0.011395379703615, 0.0171757426191143, -0.00472689955760422, 0.0138546197303906, -0.017282441148748, 0.010248414491368, 0.0260626285757702, -0.00280576641150291, 0.00102118977472798, 0, -0.023232846722337, 0.0111674969191731, -0.0214051503199956, 0.0118034157248844, -0.0089052448121078, -0.0151096313917661, -0.00858374369139144, -0.00648650922960677, 0.00890814367013811, -0.00945059011685392, 0.00189727662279466, -0.00325468147354204, -0.0189228188421451, 0.00276472390648269, -0.000828614879616203, 0.0196996794662562, -0.000542005433322949, -0.00325821629540358, -0.0214411732222413, 0.00664638281604644, -0.00859326147298701, 0.00222469502211111, -0.00892863074430132, -0.0059030389262782, -0.00622173952725283, ],
    [0.00303785911242742, 0.00151547582990027, 0.0127974652583047, 0.0190092718209104, 0.0364946864301199, -0.00971090554568595, -0.0136678898182987, -0.00217154254694342, -0.00363651498288239, 0, -0.00291672801750322, 0.00364711784480473, 0.00773265466471527, -0.0290662641433834, 0.00395458691147439, -0.0111698804689666, 0.0513319004579947, 0.0269080669484678, -0.00741373647419926, -0.00140077843916497, 0.00487963349002651, -0.00115827415580574, -0.0159049272751082, 0.00164914589485562, -0.00164914589485567, -0.00733339421955797, -0.00118689660372341, 0.00213440895878598, 0.00756763572455059, 0.00843837240378185, 0.0207943441041438, 0.00956052963076143, 0.00474620545879126, -0.00045239700493063, 0, -0.00792650286859609, -0.0020450391166462, -0.0107647066423052, 0.0119018841383533, 0.011310615413212, -0.00609335250098918, -0.00408137712007475, -0.0128068063784102, 0.00779731980213844, -0.00228927653251101, -0.00344137074074704, -0.00114815814918597, -0.0066893107527231, 0.016761469094511, 0.0227331700639883, 0.00510375629970297, 0.00199266340696577, -0.00553933360593972, 0.00399085554260851, -0.020800099265282, -0.00702904793497261, 0.00363681652749544, -0.00500284917723943, -0.00022864764544903, -0.00939099877556102, ],
    [0.0231043683454177, 0.0417104885813702, -0.0095182158209761, 0.0160098290574834, -0.0172060014489138, -0.00660464724480478, -0.00301659353942568, -0.0133821948406982, 0.0121730048505591, 0, -0.00728600673093332, 0.0211041899519374, 0.0534389621555603, -0.00226500663085212, -0.00113443007061176, -0.00284171830902758, -0.0318037790537778, -0.00885222658735119, 0.00414079266603139, 0.0035356548323078, 0.00235017734495367, 0.00643465221077915, -0.00291971010333475, 0.0127834396256749, -0.0046296378987421, -0.00348634867943777, 0.00406622692125935, -0.00464846856798463, 0, -0.0128958410829249, -0.00236267091597039, 0.0117579249936131, 0.020822036241863, 0.0136443276487771, 0, -0.0125001627642315, -0.00401031760615392, -0.00114876520387337, 0.0102916860365475, 0.00454030291878301, -0.0205956936720807, 0.0114943794257352, -0.0207859677598346, 0.0098694560184841, -0.0174829627803475, 0.00702990592825815, -0.00175284882741436, 0.0110498361865849, -0.00405680069561443, -0.00816331063916098, -0.0052832526765831, 0.0157667499714412, 0.018370206601749, -0.000568990058025087, -0.0247779890293721, -0.0117372239457254, 0.00647251450561752, -0.00176108058429767, -0.00944517057151865, 0.00414324357133459, ],
]).T
n = assets_obs.shape[1]
P = np.asmatrix(np.mean(assets_obs, axis=0))
C = np.cov(assets_obs.T)

# use APOPT to find a solition
m.options.solver = 1

u = m.Array(m.Var, n, lb=0, ub=1, integer=True)
v = m.Array(m.Var, n, lb=0, ub=1, integer=True)
y = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# lp = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# lm = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# sp = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# sm = m.Array(m.Var, n, lb=0, ub=1, integer=True)

# p = m.Const(0.001)
# for i in p:
#     i.value = 0.001

l_plus = m.Array(m.Var, n)
for i in l_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

l_minus = m.Array(m.Var, n)
for i in l_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_plus = m.Array(m.Var, n)
for i in s_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_minus = m.Array(m.Var, n)
for i in s_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1

# w_plus = m.Array(m.Var, n)
# for i in range(len(w_plus)):
#     w_plus[i].value = 0
#     w_plus[i].lower = 0.05*u[i]
#     w_plus[i].upper = 0.2*u[i]

# w_minus = m.Array(m.Var, n)
# for i in range(len(w_minus)):
#     w_minus[i].value = 0
#     w_minus[i].lower = 0.05*v[i]
#     w_minus[i].upper = 0.2*v[i]



for i in range(n):
    m.Maximize(P.item(i) * ((l_plus[i] - l_minus[i]) - (s_plus[i] - s_minus[i])))
    m.Minimize( 0.001*l_plus[i] +0.001*l_minus[i]+ 0.001*s_plus[i]+0.001*s_minus[i])
#     m.Minimize((w_plus[i] - w_minus[i]) ** 2 * C.item(i, i) ** 2)
#     for j in range(n):
#         if i != j:
#             m.Minimize(C.item(i, j) * (w_plus[i] - w_minus[i]) * (w_plus[j] - w_minus[j]))
    m.Minimize(s_plus[i]-s_minus[i])
#     m.Minimize(0.0025 * l_plus[i] + 0.0025 * l_minus[i])
#     m.Minimize(p[0] * l_plus[i] + p[1] * l_minus[i] + p[2] * s_plus[i] + p[3] * s_minus[i])


m.Equation(m.sum([(l_plus[i] - l_minus[i] +s_plus[i] - s_minus[i]\
                  + 0.001 * l_plus[i] + 0.001 * l_minus[i] \
                  + 0.001 * s_plus[i] + 0.001 * s_minus[i]) \
                 for i in range(n)]) == 1)

# m.Equation(m.sum(
#     [(w_plus[i] + 1 * w_minus[i] \
#       + p[0] * l_plus[i] + p[1] * l_minus[i] + p[2] * s_plus[i] + p[3] * s_minus[i]) for i in
#      range(n)]) == 1)

# for i in range(n):
#     m.Equation(lp[i] + lm[i] <= 1)
#     m.Equation(sp[i] + sm[i] <= 1)
#     m.Equation(w_plus[i] == l_plus[i] - l_minus[i])
#     m.Equation(w_minus[i] == s_plus[i] - s_minus[i])
#     m.Equation(u[i] + v[i] == y[i])
#     m.Equation(u[i] + v[i] <= 1)

m.solve(disp=True,debug=0)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  6
   Constants    :  0
   Variables    :  132
   Intermediates:  0
   Connections  :  24
   Equations    :  60
   Residuals    :  60
 
 Number of state variables:    132
 Number of total equations: -  30
 Number of slack variables: -  0
 ---------------------------------------
 Degrees of freedom       :    102
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    0 Obj: -7.00E+00 Gap:  0.00E+00
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :  0.016600000000000004 sec
 Objective      :  -6.9

In [13]:
print("l_plus", [i for i in l_plus])
print("l_minus", [i for i in l_minus])

print("s_plus", [i for i in s_plus])
print("s_minus", [i for i in s_minus])

l_plus [[1.0], [1.0], [1.0]]
l_minus [[0.0], [0.0], [0.0]]
s_plus [[0.0], [0.0], [0.0]]
s_minus [[0.005005005005], [1.0], [1.0]]


In [19]:
'''
教授給的code第一段可行解, w_minus以 (SP - SM)表示, 移除w_plus
'''
assets_obs = np.array([
    [0.00907849579534513, -0.00695985093505983, 0.0084299761604594, 0.00460637330420661, -0.0166407210123416, -0.0308227697859888, -0.0159197895240186, -0.0127675605999023, 0.000901306956012888, 0, -0.00904983551991793, 0.00724640852076725, 0.00292891952088526, -0.0119951647586384, -0.00433543177644596, 0.00660971067348635, -0.00958692574332696, 0.0244688238460112, 0.0120134926488238, -0.00576626126438815, -0.00311873721265473, -0.0189194832461658, -0.0285997352356136, -0.108902370498538, -0.00996337555082483, -0.011395379703615, 0.0171757426191143, -0.00472689955760422, 0.0138546197303906, -0.017282441148748, 0.010248414491368, 0.0260626285757702, -0.00280576641150291, 0.00102118977472798, 0, -0.023232846722337, 0.0111674969191731, -0.0214051503199956, 0.0118034157248844, -0.0089052448121078, -0.0151096313917661, -0.00858374369139144, -0.00648650922960677, 0.00890814367013811, -0.00945059011685392, 0.00189727662279466, -0.00325468147354204, -0.0189228188421451, 0.00276472390648269, -0.000828614879616203, 0.0196996794662562, -0.000542005433322949, -0.00325821629540358, -0.0214411732222413, 0.00664638281604644, -0.00859326147298701, 0.00222469502211111, -0.00892863074430132, -0.0059030389262782, -0.00622173952725283, ],
    [0.00303785911242742, 0.00151547582990027, 0.0127974652583047, 0.0190092718209104, 0.0364946864301199, -0.00971090554568595, -0.0136678898182987, -0.00217154254694342, -0.00363651498288239, 0, -0.00291672801750322, 0.00364711784480473, 0.00773265466471527, -0.0290662641433834, 0.00395458691147439, -0.0111698804689666, 0.0513319004579947, 0.0269080669484678, -0.00741373647419926, -0.00140077843916497, 0.00487963349002651, -0.00115827415580574, -0.0159049272751082, 0.00164914589485562, -0.00164914589485567, -0.00733339421955797, -0.00118689660372341, 0.00213440895878598, 0.00756763572455059, 0.00843837240378185, 0.0207943441041438, 0.00956052963076143, 0.00474620545879126, -0.00045239700493063, 0, -0.00792650286859609, -0.0020450391166462, -0.0107647066423052, 0.0119018841383533, 0.011310615413212, -0.00609335250098918, -0.00408137712007475, -0.0128068063784102, 0.00779731980213844, -0.00228927653251101, -0.00344137074074704, -0.00114815814918597, -0.0066893107527231, 0.016761469094511, 0.0227331700639883, 0.00510375629970297, 0.00199266340696577, -0.00553933360593972, 0.00399085554260851, -0.020800099265282, -0.00702904793497261, 0.00363681652749544, -0.00500284917723943, -0.00022864764544903, -0.00939099877556102, ],
    [0.0231043683454177, 0.0417104885813702, -0.0095182158209761, 0.0160098290574834, -0.0172060014489138, -0.00660464724480478, -0.00301659353942568, -0.0133821948406982, 0.0121730048505591, 0, -0.00728600673093332, 0.0211041899519374, 0.0534389621555603, -0.00226500663085212, -0.00113443007061176, -0.00284171830902758, -0.0318037790537778, -0.00885222658735119, 0.00414079266603139, 0.0035356548323078, 0.00235017734495367, 0.00643465221077915, -0.00291971010333475, 0.0127834396256749, -0.0046296378987421, -0.00348634867943777, 0.00406622692125935, -0.00464846856798463, 0, -0.0128958410829249, -0.00236267091597039, 0.0117579249936131, 0.020822036241863, 0.0136443276487771, 0, -0.0125001627642315, -0.00401031760615392, -0.00114876520387337, 0.0102916860365475, 0.00454030291878301, -0.0205956936720807, 0.0114943794257352, -0.0207859677598346, 0.0098694560184841, -0.0174829627803475, 0.00702990592825815, -0.00175284882741436, 0.0110498361865849, -0.00405680069561443, -0.00816331063916098, -0.0052832526765831, 0.0157667499714412, 0.018370206601749, -0.000568990058025087, -0.0247779890293721, -0.0117372239457254, 0.00647251450561752, -0.00176108058429767, -0.00944517057151865, 0.00414324357133459, ],
]).T
n = assets_obs.shape[1]
P = np.asmatrix(np.mean(assets_obs, axis=0))
C = np.cov(assets_obs.T)

# use APOPT to find a solition
m.options.solver = 1

u = m.Array(m.Var, n, lb=0, ub=1, integer=True)
v = m.Array(m.Var, n, lb=0, ub=1, integer=True)
y = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# lp = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# lm = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# sp = m.Array(m.Var, n, lb=0, ub=1, integer=True)
# sm = m.Array(m.Var, n, lb=0, ub=1, integer=True)

# p = m.Const(0.001)
# for i in p:
#     i.value = 0.001

l_plus = m.Array(m.Var, n)
for i in l_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

l_minus = m.Array(m.Var, n)
for i in l_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_plus = m.Array(m.Var, n)
for i in s_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_minus = m.Array(m.Var, n)
for i in s_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1

# w_plus = m.Array(m.Var, n)
# for i in range(len(w_plus)):
#     w_plus[i].value = 0
#     w_plus[i].lower = 0.05*u[i]
#     w_plus[i].upper = 0.2*u[i]

# w_minus = m.Array(m.Var, n)
# for i in range(len(w_minus)):
#     w_minus[i].value = 0
#     w_minus[i].lower = 0.05*v[i]
#     w_minus[i].upper = 0.2*v[i]



for i in range(n):
    m.Maximize(P.item(i) * (-s_plus[i] + s_minus[i]))
    m.Minimize( 0.001*l_plus[i] +0.001*l_minus[i]+ 0.001*s_plus[i]+0.001*s_minus[i])
#     m.Minimize((w_plus[i] - w_minus[i]) ** 2 * C.item(i, i) ** 2)
#     for j in range(n):
#         if i != j:
#             m.Minimize(C.item(i, j) * (w_plus[i] - w_minus[i]) * (w_plus[j] - w_minus[j]))
    m.Minimize(s_plus[i]-s_minus[i])
#     m.Minimize(0.0025 * l_plus[i] + 0.0025 * l_minus[i])
#     m.Minimize(p[0] * l_plus[i] + p[1] * l_minus[i] + p[2] * s_plus[i] + p[3] * s_minus[i])



m.Equation(m.sum([(s_plus[i] - s_minus[i]\
                  + 0.001 * l_plus[i] + 0.001 * l_minus[i] \
                  + 0.001 * s_plus[i] + 0.001 * s_minus[i]) \
                 for i in range(n)]) == 1)
# m.Equation(m.sum(
#     [(w_plus[i] + 1 * w_minus[i] \
#       + p[0] * l_plus[i] + p[1] * l_minus[i] + p[2] * s_plus[i] + p[3] * s_minus[i]) for i in
#      range(n)]) == 1)

# for i in range(n):
#     m.Equation(lp[i] + lm[i] <= 1)
#     m.Equation(sp[i] + sm[i] <= 1)
#     m.Equation(w_plus[i] == l_plus[i] - l_minus[i])
#     m.Equation(w_minus[i] == s_plus[i] - s_minus[i])
#     m.Equation(u[i] + v[i] == y[i])
#     m.Equation(u[i] + v[i] <= 1)

m.solve(disp=True,debug=0)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  8
   Constants    :  0
   Variables    :  182
   Intermediates:  0
   Connections  :  32
   Equations    :  86
   Residuals    :  86
 
 Number of state variables:    182
 Number of total equations: -  40
 Number of slack variables: -  0
 ---------------------------------------
 Degrees of freedom       :    142
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    4 Dpth:    0 Lvs:    0 Obj: -5.01E+00 Gap:  0.00E+00
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :  0.018900000000000028 sec
 Objective      :  -5.0

In [20]:
print("l_plus", [i for i in l_plus])
print("l_minus", [i for i in l_minus])

print("s_plus", [i for i in s_plus])
print("s_minus", [i for i in s_minus])

l_plus [[1.0], [1.0], [1.0]]
l_minus [[1.0], [1.0], [1.0]]
s_plus [[1.0], [0.0], [1.0]]
s_minus [[0.0], [1.0], [0.009009009009]]


In [24]:
'''
    只修改 m.Minimize(s_plus[i]+s_minus[i]) -> m.Minimize(s_plus[i]-s_minus[i])
'''

assets_obs = np.array([
    [0.00907849579534513, -0.00695985093505983, 0.0084299761604594, 0.00460637330420661, -0.0166407210123416, -0.0308227697859888, -0.0159197895240186, -0.0127675605999023, 0.000901306956012888, 0, -0.00904983551991793, 0.00724640852076725, 0.00292891952088526, -0.0119951647586384, -0.00433543177644596, 0.00660971067348635, -0.00958692574332696, 0.0244688238460112, 0.0120134926488238, -0.00576626126438815, -0.00311873721265473, -0.0189194832461658, -0.0285997352356136, -0.108902370498538, -0.00996337555082483, -0.011395379703615, 0.0171757426191143, -0.00472689955760422, 0.0138546197303906, -0.017282441148748, 0.010248414491368, 0.0260626285757702, -0.00280576641150291, 0.00102118977472798, 0, -0.023232846722337, 0.0111674969191731, -0.0214051503199956, 0.0118034157248844, -0.0089052448121078, -0.0151096313917661, -0.00858374369139144, -0.00648650922960677, 0.00890814367013811, -0.00945059011685392, 0.00189727662279466, -0.00325468147354204, -0.0189228188421451, 0.00276472390648269, -0.000828614879616203, 0.0196996794662562, -0.000542005433322949, -0.00325821629540358, -0.0214411732222413, 0.00664638281604644, -0.00859326147298701, 0.00222469502211111, -0.00892863074430132, -0.0059030389262782, -0.00622173952725283, ],
    [0.00303785911242742, 0.00151547582990027, 0.0127974652583047, 0.0190092718209104, 0.0364946864301199, -0.00971090554568595, -0.0136678898182987, -0.00217154254694342, -0.00363651498288239, 0, -0.00291672801750322, 0.00364711784480473, 0.00773265466471527, -0.0290662641433834, 0.00395458691147439, -0.0111698804689666, 0.0513319004579947, 0.0269080669484678, -0.00741373647419926, -0.00140077843916497, 0.00487963349002651, -0.00115827415580574, -0.0159049272751082, 0.00164914589485562, -0.00164914589485567, -0.00733339421955797, -0.00118689660372341, 0.00213440895878598, 0.00756763572455059, 0.00843837240378185, 0.0207943441041438, 0.00956052963076143, 0.00474620545879126, -0.00045239700493063, 0, -0.00792650286859609, -0.0020450391166462, -0.0107647066423052, 0.0119018841383533, 0.011310615413212, -0.00609335250098918, -0.00408137712007475, -0.0128068063784102, 0.00779731980213844, -0.00228927653251101, -0.00344137074074704, -0.00114815814918597, -0.0066893107527231, 0.016761469094511, 0.0227331700639883, 0.00510375629970297, 0.00199266340696577, -0.00553933360593972, 0.00399085554260851, -0.020800099265282, -0.00702904793497261, 0.00363681652749544, -0.00500284917723943, -0.00022864764544903, -0.00939099877556102, ],
    [0.0231043683454177, 0.0417104885813702, -0.0095182158209761, 0.0160098290574834, -0.0172060014489138, -0.00660464724480478, -0.00301659353942568, -0.0133821948406982, 0.0121730048505591, 0, -0.00728600673093332, 0.0211041899519374, 0.0534389621555603, -0.00226500663085212, -0.00113443007061176, -0.00284171830902758, -0.0318037790537778, -0.00885222658735119, 0.00414079266603139, 0.0035356548323078, 0.00235017734495367, 0.00643465221077915, -0.00291971010333475, 0.0127834396256749, -0.0046296378987421, -0.00348634867943777, 0.00406622692125935, -0.00464846856798463, 0, -0.0128958410829249, -0.00236267091597039, 0.0117579249936131, 0.020822036241863, 0.0136443276487771, 0, -0.0125001627642315, -0.00401031760615392, -0.00114876520387337, 0.0102916860365475, 0.00454030291878301, -0.0205956936720807, 0.0114943794257352, -0.0207859677598346, 0.0098694560184841, -0.0174829627803475, 0.00702990592825815, -0.00175284882741436, 0.0110498361865849, -0.00405680069561443, -0.00816331063916098, -0.0052832526765831, 0.0157667499714412, 0.018370206601749, -0.000568990058025087, -0.0247779890293721, -0.0117372239457254, 0.00647251450561752, -0.00176108058429767, -0.00944517057151865, 0.00414324357133459, ],
]).T
n = assets_obs.shape[1]

P = np.asmatrix(np.mean(assets_obs, axis=0))
# w = np.asmatrix(np.random.dirichlet(np.ones(n), size=1))
C = np.cov(assets_obs.T)

# use APOPT to find a solition
m.options.solver = 1


u = m.Array(m.Var, n, lb=0, ub=1, integer=True)
v = m.Array(m.Var, n, lb=0, ub=1, integer=True)
y = m.Array(m.Var, n, lb=0, ub=1, integer=True)
lp = m.Array(m.Var, n, lb=0, ub=1, integer=True)
lm = m.Array(m.Var, n, lb=0, ub=1, integer=True)
sp = m.Array(m.Var, n, lb=0, ub=1, integer=True)
sm = m.Array(m.Var, n, lb=0, ub=1, integer=True)

# p = m.Const(0.001)
# for i in p:
#     i.value = 0.001

l_plus = m.Array(m.Var, n)
for i in l_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

l_minus = m.Array(m.Var, n)
for i in l_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_plus = m.Array(m.Var, n)
for i in s_plus:
    i.value = 0
    i.lower = 0
    i.upper = 1

s_minus = m.Array(m.Var, n)
for i in s_minus:
    i.value = 0
    i.lower = 0
    i.upper = 1

# w_plus = m.Array(m.Var, n)
# for i in range(len(w_plus)):
#     w_plus[i].value = 0
#     w_plus[i].lower = 0.05*u[i]
#     w_plus[i].upper = 0.2*u[i]

# w_minus = m.Array(m.Var, n)
# for i in range(len(w_minus)):
#     w_minus[i].value = 0
#     w_minus[i].lower = 0.05*v[i]
#     w_minus[i].upper = 0.2*v[i]

# l_plus = m.Array(m.Var, n)
# for i in range(len(l_plus)):
#     l_plus[i].value = 0
#     l_plus[i].lower = 0*lp[i]
#     l_plus[i].upper = 1*lp[i]

# l_minus = m.Array(m.Var, n)
# for i in range(len(l_minus)):
#     l_minus[i].value = 0
#     l_minus[i].lower = 0*lm[i]
#     l_minus[i].upper = 1*lm[i]

# s_plus = m.Array(m.Var, n)
# for i in range(len(s_plus)):
#     s_plus[i].value = 0
#     s_plus[i].lower = 0*sp[i]
#     s_plus[i].upper = 1*sp[i]

# s_minus = m.Array(m.Var, n)
# for i in range(len(s_minus)):
#     s_minus[i].value = 0
#     s_minus[i].lower = 0*sm[i]
#     s_minus[i].upper = 1*sm[i]
    

for i in range(n):
    m.Maximize(P.item(i) * (l_plus[i] -s_plus[i]))
    m.Minimize( 0.001*l_plus[i] +0.001*l_minus[i]+ 0.001*s_plus[i]+0.001*s_minus[i])
#     m.Minimize((w_plus[i] - w_minus[i]) ** 2 * C.item(i, i) ** 2)
#     for j in range(n):
#         if i != j:
#             m.Minimize(C.item(i, j) * (w_plus[i] - w_minus[i]) * (w_plus[j] - w_minus[j]))
    m.Minimize(s_plus[i]-s_minus[i])
#     m.Minimize(0.0025 * l_plus[i] + 0.0025 * l_minus[i])
#     m.Minimize(p[0] * l_plus[i] + p[1] * l_minus[i] + p[2] * s_plus[i] + p[3] * s_minus[i])


m.Equation(m.sum([(l_plus[i] + l_minus[i] +s_plus[i] + s_minus[i]\
                  + 0.001 * l_plus[i] + 0.001 * l_minus[i] \
                  + 0.001 * s_plus[i] + 0.001 * s_minus[i]) \
                 for i in range(n)]) == 1)

# m.Equation(m.sum(
#     [(w_plus[i] + 1 * w_minus[i] \
#       + p[0] * l_plus[i] + p[1] * l_minus[i] + p[2] * s_plus[i] + p[3] * s_minus[i]) for i in
#      range(n)]) == 1)

# for i in range(n):
#     m.Equation(lp[i] + lm[i] <= 1)
#     m.Equation(sp[i] + sm[i] <= 1)
#     m.Equation(w_plus[i] == l_plus[i] - l_minus[i])
#     m.Equation(w_minus[i] == s_plus[i] - s_minus[i])
#     m.Equation(u[i] + v[i] == y[i])
#     m.Equation(u[i] + v[i] <= 1)

m.solve(disp=True,debug=0)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.0
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :  10
   Constants    :  0
   Variables    :  256
   Intermediates:  0
   Connections  :  40
   Equations    :  112
   Residuals    :  112
 
 Number of state variables:    256
 Number of total equations: -  50
 Number of slack variables: -  0
 ---------------------------------------
 Degrees of freedom       :    206
 
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.00 NLPi:    3 Dpth:    0 Lvs:    0 Obj: -7.01E+00 Gap:  0.00E+00
 Successful solution
 
 ---------------------------------------------------
 Solver         :  APOPT (v1.0)
 Solution time  :  0.016000000000000014 sec
 Objective      :  -

In [25]:
print("l_plus", [i for i in l_plus])
print("l_minus", [i for i in l_minus])
print("s_plus", [i for i in s_plus])
print("s_minus", [i for i in s_minus])

l_plus [[0.0], [0.0], [0.0]]
l_minus [[0.0], [0.0], [0.0]]
s_plus [[0.0], [0.0], [0.0]]
s_minus [[0.33300033302], [0.33300033302], [0.33300033297]]
